In [ ]:
project = 'saga-veglogg-prod-wznf'
use_colab_auth = True

# Legg inn ditt eget prosjekt her, f.eks. 'saga-olanor-playground-ab12'
bq_job_project = ''

In [ ]:
if (use_colab_auth):
  from google.colab import auth
  auth.authenticate_user()
  print('Authenticated')

In [ ]:
import warnings
from google.cloud import bigquery

warnings.filterwarnings('ignore')
client = bigquery.Client(project=bq_job_project)

Denne spørringen viser antall hendelser fra vegloggen i desember 2021 per fylkesnummer.

In [ ]:
query = f"""
SELECT
    COALESCE(countyNumber, 'Ukjent') fylkesNr,
    COUNT(*) antallHendelser
FROM `{project}.standardized.situations`
WHERE DATE(creationTime, "Europe/Oslo") BETWEEN "2021-12-01" AND "2021-12-31"
GROUP BY countyNumber
ORDER BY antallHendelser DESC
"""

print(query)

client.query(query).to_dataframe()

Denne spørringen viser antall hendelser fra vegloggen i desember 2021 per vegnummer, for hendelser innen 10 km av Trondheim sentrum.

In [ ]:
query = f"""
SELECT
    roadNumber vegNr,
    COUNT(*) antallHendelser,
FROM `{project}.standardized.situations`
WHERE DATE(creationTime, "Europe/Oslo") BETWEEN "2021-12-01" AND "2021-12-31"
    AND roadNumber IS NOT NULL
    AND st_distance(geography, st_geogfromtext('POINT (10.3939341 63.4301525)')) < 10000
GROUP BY vegNr
ORDER BY antallHendelser DESC
"""

print(query)

client.query(query).to_dataframe()